<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/multi_material-lungman_phantom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Multi-material example: Lungman phantom

In this notebook you will use the knowledge learned in [first_xray_simulation.ipynb](first_xray_simulation.ipynb) to set some of the simulation parameters. 
We aim to **replicate a digital radiograph (DR)** of the [Lungman anthropomorphic chest phantom](https://doi.org/10.1117/1.JMI.5.1.013504) (Kyoto Kagaku, Tokyo, Japan) taken with a clinical X-ray machine by GE Healthcare (Chicago, Illinois, USA) at one of our local hospitals (Glan Clwyd). Parameters relevant to the simulation are extracted from the DICOM file, such as source-patient-distance and source-detector-distance.

![Photograph of the Lungman phantom during the digital radiograph acquisition](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/lungman-photo.jpg?raw=1)

Additionaly, we will **demonstrate how to use more than one material**. 
We scanned the [Lungman anthropomorphic chest phantom](https://doi.org/10.1117/1.JMI.5.1.013504) at the same hospital (Ysbyty Gwynedd) using a 128-slice Somatom Definition Edge scanner by Siemens Healthcare (Erlangen, Germany). 
The CT volume was then segmented into individual structures. 
Surface meshes were eventually extracted from the segmented CT scan. 
The corresponding data is available on [Zenodo](https://zenodo.org/records/10782644) (the notebook will download and extract the files automatically). 
The material property of each anatomical structure is defined as the average Hounsfield unit of that structure. 
Hounsfield values are then converted into material compositions (mixtures) and densities using [Schneider et al.'s method](https://doi.org/10.1088/0031-9155/45/2/314). This functionality is already built in gVXR.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the required packages.
</div>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gvxr SimpleITK k3d

## Aims of this session

1. Practice, amongst other things, how to set simulation parameters related to the X-ray source and detector;
2. Demonstrate how to load several STL files and use them all in the simulation (multi-material).

![Screenshot of the 3D environment using K3D](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/output_data/lungman/k3d_screenshot-cropped.png?raw=1)

## Summary of the simulation parameters

| Parameters | Values | Units | Function call |
|------------|--------|-------|---------------|
| Source to patient distance | 1751 | mm | `gvxr.setSourcePosition(...)` |
| Source type (beam shape) | Point source (cone beam) | | `gvxr.usePointSource()` |
| Beam spectrum | Monochromatic: 16,000 photons of 90 | keV | `gvxr.setMonoChromatic(...)` |
| Source to detector distance | 1800 | mm | `gvxr.setDetectorPosition(...)` |
| Detector orientation | [0, 0, -1] |  | `gvxr.setDetectorUpVector(...)` |
| Detector resolution | 1871 &times; 1881 | pixels | `gvxr.setDetectorNumberOfPixels(...)` |
| Pixel spacing | 0.2 &times; 0.2 | mm | `gvxr.setDetectorPixelSize(...)` |

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `SimpleITK` to load the DICOM file
- `gvxr` to simulate X-ray images

In [ ]:
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

import SimpleITK as sitk

import urllib.request
import progressbar

import zipfile
import base64

from gvxrPython3 import gvxr
from gvxrPython3.utils import visualise

## Getting the data ready

In [ ]:
output_path = "output_data/multi_material-lungman_phantom"
if not os.path.exists(output_path):
    os.makedirs(output_path);

lungman_path = os.path.join("output_data", "lungman");
mesh_path = os.path.join(lungman_path, "MESHES");
if not os.path.exists(mesh_path):
    os.makedirs(mesh_path);    

Download the data from [Zenodo](https://zenodo.org/records/10782644).

In [ ]:
zip_file_url = "https://zenodo.org/records/10782644/files/lungman_data.zip?download=1";

zip_fname = os.path.join(lungman_path, "lungman_data.zip");

if not os.path.exists(zip_fname):
    pbar = None;
    def show_progress(block_num, block_size, total_size):
        global pbar
        if pbar is None:
            pbar = progressbar.ProgressBar(maxval=total_size)
            pbar.start()
    
        downloaded = block_num * block_size
        if downloaded < total_size:
            pbar.update(downloaded)
        else:
            pbar.finish()
            pbar = None
            
    print("Download the file (%s) from %s\n" % (zip_fname, zip_file_url))
    urllib.request.urlretrieve(zip_file_url, zip_fname, show_progress)

Extract the STL files from the ZIP file.

In [ ]:
stl_fname_set = [];

with zipfile.ZipFile(zip_fname) as z:
    for fname in z.namelist():
        if ".stl" in fname:
            stl_fname = os.path.join(lungman_path, fname);
            stl_fname_set.append(stl_fname);
            
            if not os.path.exists(stl_fname):
                print("Extract %s" % stl_fname);
                with open(stl_fname, 'wb') as f:
                    f.write(z.read(fname));

Extract the digital radiograph DICOM file from the ZIP file.

In [ ]:
input_DICOM_fname = "CD3/DICOM/ST000000/SE000000/DX000000"
output_DICOM_fname = os.path.join(lungman_path, "DX000000");

with zipfile.ZipFile(zip_fname) as z:
    if not os.path.exists(output_DICOM_fname):
        print("Extract %s" % input_DICOM_fname);
        with open(output_DICOM_fname, 'wb') as f:
            f.write(z.read(input_DICOM_fname));

Read the DICOM file.

In [ ]:
reader = sitk.ImageFileReader();
reader.SetImageIO("GDCMImageIO");
reader.SetFileName(output_DICOM_fname);
reader.LoadPrivateTagsOn();
reader.ReadImageInformation();    
volume = reader.Execute();
raw_reference = sitk.GetArrayFromImage(volume)[0];

Extract information useful for the simulation from the DICOM file:

- The image size and the physical pixel spacing (i.e. not taking into account the magnification),
- The number of pixels,
- The distance from the source to the detector, and
- The distance from the source to the patient.

In [ ]:
# Extract the information from the DICOM header
imager_pixel_spacing = np.array(volume.GetMetaData("0018|1164").split("\\")).astype(np.single);
detector_element_spacing = np.array(volume.GetMetaData("0018|7022").split("\\")).astype(np.single);
print("Imager Pixel Spacing (in mm): ", imager_pixel_spacing, "(with magnification)");
print("Detector Element Spacing (in mm): ", detector_element_spacing, "(without magnification)");

# Extract the number of pixels
size = volume.GetSize()[0:2]
print("Image size (in pixels): ", str(size[0]) + " x " + str(size[1]))

# Extract the information from the DICOM header
distance_source_to_detector = float(volume.GetMetaData("0018|1110"))
distance_source_to_patient = float(volume.GetMetaData("0018|1111"))

print("Distance Source to Detector: ", distance_source_to_detector, "mm")
print("Distance Source to Patient: ", distance_source_to_patient, "mm")

We also extract the visualisation window to show the image using the 'harder' window.

In [ ]:
window_centre = int(volume.GetMetaData("0028|1050").split("\\")[1]) # Use 0 for normal, 1 for harder, 2 for softer
window_width = int(volume.GetMetaData("0028|1051").split("\\")[1]) # Use 0 for normal, 1 for harder, 2 for softer

print("Window Center used: ", window_centre)
print("Window Width used: ", window_width)

vmin = window_centre - window_width / 2
vmax = window_centre + window_width / 2

view_position = volume.GetMetaData("0018|5101")

In [ ]:
fig = plt.figure();
plt.imshow(raw_reference, cmap="gray", vmin=vmin, vmax=vmax,
                             extent=[0,(raw_reference.shape[1]-1)*imager_pixel_spacing[0],0,(raw_reference.shape[0]-1)*imager_pixel_spacing[1]])
plt.title("Digital radiograph of the Lungman phantom")
plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.colorbar()
plt.show()

## 1. Create an OpenGL context

The first step is to create the simulation environment, known here as "OpenGL context".
`gvxr.createOpenGLContext` will try to find the most suitable environment possible regardless of the operating system. This is an alternative function to `gvxr.createNewContext` used in [test_installation.ipynb](test_installation.ipynb).

---
## Task:

In the cell below, call `gvxr.createOpenGLContext` or `gvxr.createNewContext` to create the simulation environment.

In [ ]:
# %load 'snippets/multi_material-lungman_phantom-01.py'

We increase the size of the visualisation framebuffer to generate higher resolution screenshots. It does not affect the simulation.

In [ ]:
gvxr.setWindowSize(1000, 1000)

## 2. Set the X-ray source

We must set it's position and beam shape. 

- The distance from the source to the patient is 1751.0 mm.
- As we are trying to replicate an actual acquisition, a cone-beam geometry should be used.

---
## Task:

In the cell below, 
- set the source position so that the distance from the source to the patient is 1751.0 mm. Consider that the patient position is in [0, 0, 0];
- make sure a point source is used.

In [ ]:
# %load 'snippets/multi_material-lungman_phantom-02.py'

## 3. Set the Spectrum

We define here the number of photons and their kinetic energy.

---
## Task:

In the cell below, use a monochromatic source that emits 16,000 photons of 90 keV per pixel.

In [ ]:
# %load 'snippets/multi_material-lungman_phantom-03.py'

## 4. Set the Detector

A detector is defined by its position, orientation, pixel resolution and the space between the centre of two consecutive pixels along its two axes.

---
## Task:

- Use the detector position.
    - distance from the source to the patient is 1751.0 mm,
    - distance from the source to the detector is 1800.0 mm,
    - the patient position is in [0, 0, 0],
    - therefore the distance from the patient to the detector is 1800 minus 1751 mm.
- The up vector defining the detector orientation is (0, 0, 1).
- There are 1871 &times; 1881 pixels.
- The pixel spacing in mm is [0.2, 0.2].

In [ ]:
# %load 'snippets/multi_material-lungman_phantom-04.py'

## 5. Set the Sample

A sample is define by its geometry (surface) and material composition. Note that you can transform (translate, scale and rotate) a sample.

In [ ]:
gvxr.removePolygonMeshesFromXRayRenderer();

gvxr.emptyMesh("lungman");

geometry_set = {
    "bronchioles": {"HU": -419.57144, "Colour" : [0, 240, 240, 0.08]},
    "bronchus": {"HU": -40.36795, "Colour" : [0, 62, 186, 0.4]},
    "trachea": {"HU": -914.32916, "Colour" : [170, 85, 255, 0.4]},
    "diaphram": {"HU": -12.778751, "Colour" : [255, 85, 127, 1]},
    "skin": {"HU": -12.121676, "Colour" : [125, 125, 125, 0.17]},
    "heart": {"HU": 28.384626, "Colour" : [255, 0, 0, 1]},
    "sheets_low": {"HU": -158.2706, "Colour" : [193, 193, 193, 1]},
    "sheets_med": {"HU": 203.39578, "Colour" : [193, 193, 193, 1]},
    "sheets_high": {"HU": 324.9135, "Colour" : [193, 193, 193, 1]},
    "tumours_630HU": {"HU": -658.61346, "Colour" : [138, 0, 0, 1]},
    "tumours_100HU": {"HU": 83.32481, "Colour" : [255, 85, 0, 1]},
    "spine-hard-650": {"HU": 857.8602, "Colour" : [255, 255, 127, 1]},
    "spine-soft-650": {"HU": 375.58865, "Colour" : [255, 255, 127, 1]},
    "scaps-hard-550": {"HU": 709.09717, "Colour" : [255, 255, 127, 1]},
    "scaps-soft-550": {"HU": 372.82138, "Colour" : [255, 255, 127, 1]},
    "sternum-hard-550": {"HU": 789.6037, "Colour" : [255, 255, 127, 1]},
    "sternum-soft-550": {"HU": 378.79736, "Colour" : [255, 255, 127, 1]},
    "clavicle-hard-700": {"HU": 778.28, "Colour" : [255, 255, 127, 1]},
    "clavicle-soft-700": {"HU": 261.89047, "Colour" : [255, 255, 127, 1]},
}

for label in geometry_set:
    if "sheet" not in label:
        gvxr.loadMeshFile(label, os.path.join(mesh_path, label + ".stl"), "mm", True, "root");
        gvxr.setHounsfieldUnit(label, round(geometry_set[label]["HU"]));
        gvxr.setColour(label,
            geometry_set[label]["Colour"][0] / 255.0,
            geometry_set[label]["Colour"][1] / 255.0,
            geometry_set[label]["Colour"][0] / 255.0,
            geometry_set[label]["Colour"][3]);

        # We translate the geometry to make sure the detector is not within the patient
        gvxr.translateNode(label, 0, -118, 0, "mm")

## 6. Compute the corresponding X-ray image.

It is possible to compute, retrieve and save an X-ray image as well as the path length of X-ray through an object.

---
## Tasks:

- Compute an X-ray image and store it in a local variable called `x_ray_image`
- Make sure to convert it into a Numpy array, and
- Divide the pixel values by `gvxr.getTotalEnergyWithDetectorResponse()` (mock flat-field correction, this way the pixels values with be in the range [0, 1])

In [ ]:
# %load 'snippets/multi_material-lungman_phantom-05.py'

## Visualisation of the results

Keep in mind that clinical devices implement built-in image post-processing algorithms that change the pixel values. Specifics of these algorithms are unknown. We post-process our simulated X-ray image to display the image in a similar fashion using:

$$(-\log((Img + shift_1) \times scale_1) + shift_2) \times scale_2$$

Values of $shift_1$, $scale_1$, $shift_2$ and $scale_2$ have been calibrated.

In [ ]:
# Crop the image
y_min_id = 200
y_max_id = raw_reference.shape[0] - 250
x_min_id = 50
x_max_id = raw_reference.shape[1] - 100

cropped_x_ray_image = x_ray_image[y_min_id:y_max_id, x_min_id:x_max_id]
cropped_raw_reference = raw_reference[y_min_id:y_max_id, x_min_id:x_max_id]

# Change the dynamic range to match the one of the experimental data
log_epsilon = 1e-9;
cropped_x_ray_image += 1.955133958883307132e-04;
cropped_x_ray_image *= 9.301220532307965186e+03;
cropped_x_ray_image[cropped_x_ray_image < log_epsilon] = log_epsilon;
cropped_x_ray_image = -np.log(cropped_x_ray_image);
cropped_x_ray_image += 1.451272580374132559e+01;
cropped_x_ray_image *= 3.990621875946345654e+00;

mean_test = np.mean(cropped_x_ray_image)
stddev_test = np.std(cropped_x_ray_image)

mean_ref = np.mean(cropped_raw_reference)
stddev_ref = np.std(cropped_raw_reference)

cropped_x_ray_image -= mean_test
cropped_x_ray_image /= stddev_test

cropped_x_ray_image *= stddev_ref
cropped_x_ray_image += mean_ref

In [ ]:
# Plot the two images side-by-side
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))

im1 = axes.flat[0].imshow(cropped_raw_reference, cmap="gray", vmin=vmin, vmax=vmax,
    extent=[0,(cropped_raw_reference.shape[1]-1)*imager_pixel_spacing[0],0,(cropped_raw_reference.shape[0]-1)*imager_pixel_spacing[1]])
axes.flat[0].set_title("Ground truth")


im2 = axes.flat[1].imshow(cropped_x_ray_image, cmap="gray", vmin=vmin, vmax=vmax,
    extent=[0,(cropped_x_ray_image.shape[1]-1)*imager_pixel_spacing[0],0,(cropped_x_ray_image.shape[0]-1)*imager_pixel_spacing[1]])
axes.flat[1].set_title("gVirtualXRay")

axes.flat[0].set_xlabel("Pixel position\n(in mm)")
axes.flat[1].set_xlabel("Pixel position\n(in mm)")
axes.flat[0].set_ylabel("Pixel position\n(in mm)")

cb_ax = fig.add_axes([0.925, 0.35, 0.02, 0.275])
cbar = fig.colorbar(im1, cax=cb_ax)
plt.show()

It may be useful to visualise the 3D environment to ascertain everything is as expected. It can be done in a window or offscreen with a screenshot.
We use both the functionality built in gVXR and K3D

In [ ]:
def cropImage(img):
    image_data_greyscale = np.array(img).max(axis=2)

    non_empty_columns = np.where(image_data_greyscale.min(axis=0)<255)[0]
    non_empty_rows = np.where(image_data_greyscale.min(axis=1)<255)[0]

    # In case no cropping is possible
    if non_empty_columns.shape[0] == 0 or non_empty_rows.shape[0] == 0:
        return img

    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))
    
    return np.array(img)[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]

In [ ]:
# This image can be used in a research paper to illustrate the simulation environment, in which case you may want to change the background colour to white with:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0);

# Update the visualisation window
gvxr.displayScene();

# Take the screenshot and save it in a file
screenshot = cropImage(255 * np.array(gvxr.takeScreenshot())).astype(np.uint8);
plt.imsave(os.path.join(output_path, "screenshot-01.png"), np.array(screenshot));

# or display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

We rotate and zoom out to make a nicer screenshot.

In [ ]:
gvxr.setZoom(1900)
gvxr.setSceneRotationMatrix([
    -0.8833954334259033, 0.017649494111537933, 0.46829527616500854, 0.0,
    -0.46784770488739014, 0.024374328553676605, -0.883472204208374, 0.0, 
    -0.02700728178024292, -0.9995452165603638, -0.01327541097998619, 0.0,
    0.0, 0.0, 0.0, 1.0]);

# Update the visualisation window
gvxr.displayScene();

# Take the screenshot and save it in a file
screenshot = cropImage(255 * np.array(gvxr.takeScreenshot())).astype(np.uint8);
plt.imsave(os.path.join(output_path, "screenshot-02.png"), np.array(screenshot));

# or display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

We use k3D if possible. It's a nice 3D visualisation framework for Jupyter notebooks.

In [ ]:
plot = visualise(use_log=True, use_negative=True, sharpen_ksize=2, sharpen_alpha=1.0);

if plot:
    plot.display();

In [ ]:
if plot:
    plot.fetch_screenshot();

In [ ]:
# if plot:
#     k3d_screenshot = plot.screenshot;
#     data = base64.b64decode(k3d_screenshot);
#     with open(os.path.join(lungman_path, "k3d_screenshot.png"), "wb") as fp:
#         fp.write(data);
#         fp.flush();
#         fp.close();

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
gvxr.terminate();